### 连接ｓｐａｒｋ并创建DataFrame

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("yarn").appName("ops").getOrCreate()
df = spark.read.csv('hdfs://master:9000/dataset/appl_stock.csv', inferSchema=True, header=True)

df.take(2)

[Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039),
 Row(Date=datetime.datetime(2010, 1, 5, 0, 0), Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002)]

### 查看DataFrame模式

In [2]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



### 查看列名，行数等

In [12]:
print(df.columns)
print(df.count())
df.show(3)

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']
1762
+--------------------+----------+----------+------------------+----------+---------+------------------+
|                Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+--------------------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04 00:00:...|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05 00:00:...|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:...|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|
+--------------------+----------+----------+------------------+----------+---------+------------------+
only showing top 3 rows



### 计算某一列的数值特征

In [21]:
df.describe(['close']).show()

+-------+-----------------+
|summary|            close|
+-------+-----------------+
|  count|             1762|
|   mean|312.9270656379113|
| stddev|185.1471036170943|
|    min|        90.279999|
|    max|       702.100021|
+-------+-----------------+



### 选择和切片筛选

In [7]:
df.filter('close < 500').select(['High', 'Low']).show(5)

+----------+------------------+
|      High|               Low|
+----------+------------------+
|214.499996|212.38000099999996|
|215.589994|        213.249994|
|    215.23|        210.750004|
|212.000006|        209.050005|
|212.000006|209.06000500000002|
+----------+------------------+
only showing top 5 rows



### 对于列名大小写不敏感

In [9]:
df.filter(df['close'] < 500).select(['high', 'low']).show(5)

+----------+------------------+
|      high|               low|
+----------+------------------+
|214.499996|212.38000099999996|
|215.589994|        213.249994|
|    215.23|        210.750004|
|212.000006|        209.050005|
|212.000006|209.06000500000002|
+----------+------------------+
only showing top 5 rows



In [10]:
df.filter((df['open'] > 200) & ~(df['close'] < 200)).show(5)

+--------------------+----------+----------+------------------+------------------+---------+------------------+
|                Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+--------------------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:...|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:...|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:...|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:...|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:...|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+--------------------+----------+----------+------------------+------------------+---------+------------

In [22]:
df.filter(df['low'] == 197.16).show()

+--------------------+------------------+----------+------+------+---------+---------+
|                Date|              Open|      High|   Low| Close|   Volume|Adj Close|
+--------------------+------------------+----------+------+------+---------+---------+
|2010-01-22 00:00:...|206.78000600000001|207.499996|197.16|197.75|220441900|25.620401|
+--------------------+------------------+----------+------+------+---------+---------+



### 将DataFrame转换成RDD[Row]，然后将Row类型转换为Python字典类型

In [24]:
result = df.filter(df['low'] == 197.16).collect()
print(result)
row = result[0]
row.asDict()['Volume']

[Row(Date=datetime.datetime(2010, 1, 22, 0, 0), Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)]


220441900

In [57]:
df = spark.read.csv('hdfs://master:9000/dataset/sales_info.csv', inferSchema=True, header=True)
df.show()
df.printSchema()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [58]:
df.filter('person like "%l%"').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [59]:
df.where('person like "%l%"').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



### Groupby and Aggregate

### 跟数据库不一样，groupby不会直接出结果，只是一个transform操作

In [28]:
df.groupBy('company')

In [30]:
df.groupBy('company').mean().show()

+-------+-----------------+
|company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [31]:
df.groupBy('company').sum().show()

+-------+----------+
|company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+



In [32]:
df.groupBy('company').count().show()

+-------+-----+
|company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [33]:
df.agg({'sales': 'min'}).show()

+----------+
|min(sales)|
+----------+
|     120.0|
+----------+



In [34]:
df.agg({'sales': 'sum'}).show()

+----------+
|sum(sales)|
+----------+
|    4327.0|
+----------+



In [35]:
group_data = df.groupBy('company')
group_data.agg({'sales' : 'sum'}).show()

+-------+----------+
|company|sum(sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+



### 计算某个字段类别的数量

In [39]:
from pyspark.sql.functions import countDistinct, avg, stddev, format_number
df.select(countDistinct('sales')).show()

+---------------------+
|count(DISTINCT sales)|
+---------------------+
|                   11|
+---------------------+



### 重命名列名

In [37]:
df.select(avg('sales').alias('Average Sales')).show()

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



### 计算标准差

In [38]:
df.select(stddev('sales').alias('std')).show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



### 对数值格式化

In [41]:
sales_std = df.select(stddev('sales').alias('std'))
sales_std.select(format_number('std', 2).alias('std')).show()

+------+
|   std|
+------+
|250.09|
+------+



### 排序

In [43]:
df.orderBy('sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [44]:
df.orderBy(df['sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



### 对缺失值进行处理

In [46]:
df = spark.read.csv('hdfs://master:9000/dataset/ContainsNull.csv', inferSchema = True, header = True)
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [47]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



### thresh表示一行中非null的个数

In [49]:
df.na.drop(thresh=3).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [50]:
df.na.drop(subset='sales').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [51]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [52]:
df.na.fill('no name', subset = 'Name').show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|no name| null|
|emp3|no name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+

